In [1]:
# Backprop on the Vowel Dataset
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
import numpy as np
import csv
 

In [15]:
from google.colab import files
uploaded = files.upload()

Saving Iris.csv to Iris.csv


In [2]:
# Load a CSV file
def loadCsv(filename):
        trainSet = []
        
        lines = csv.reader(open(filename, 'r'))
        dataset = list(lines)
        for i in range(len(dataset)):
                for j in range(4):
                        #print("DATA {}".format(dataset[i]))
                        dataset[i][j] = float(dataset[i][j])
                trainSet.append(dataset[i])
        return trainSet


In [3]:
def minmax(dataset):
        minmax = list()
        stats = [[min(column), max(column)] for column in zip(dataset)]
        return stats
 
# Rescale dataset columns to the range 0-1
def normalize(dataset, minmax):
        for row in dataset:
                for i in range(len(row)-1):
                        row[i] = float ((row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0]))
 
# Convert string column to float
def column_to_float(dataset, column):
        for row in dataset:
                try:
                        row[column] = float(row[column])
                except ValueError:
                        print("Error with row",column,":",row[column])
                        pass

In [4]:
def column_to_int(dataset, column):
        class_values = [row[column] for row in dataset]
        unique = set(class_values)
        lookup = dict()
        for i, value in enumerate(unique):
                lookup[value] = i
        for row in dataset:
                row[column] = lookup[row[column]]
        return lookup

In [5]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
        dataset_split = list()
        dataset_copy = list(dataset)
        fold_size = int(len(dataset) / n_folds)
        for i in range(n_folds):
                fold = list()
                while len(fold) < fold_size:
                        index = randrange(len(dataset_copy))
                        fold.append(dataset_copy.pop(index))
                dataset_split.append(fold)
        return dataset_split
 
# Calculate accuracy percentage
def accuracy_met(actual, predicted):
        correct = 0
        for i in range(len(actual)):
                if actual[i] == predicted[i]:
                        correct += 1
        return correct / float(len(actual)) * 100.0
 

In [6]:
def run_algorithm(dataset, algorithm, n_folds, *args):
        
        folds = cross_validation_split(dataset, n_folds)
        #for fold in folds:
                #print("Fold {} \n \n".format(fold))
        scores = list()
        for fold in folds:
                #print("Test Fold {} \n \n".format(fold))
                train_set = list(folds)
                index = np.argwhere(train_set==fold)
                deleted = np.delete(train_set,index)
                #print ("deleted : ", deleted)
                train_set = sum(train_set, [])
                test_set = list()
                for row in fold:
                        row_copy = list(row)
                        test_set.append(row_copy)
                        row_copy[-1] = None
                predicted = algorithm(train_set, test_set, *args)
                actual = [row[-1] for row in fold]
                accuracy = accuracy_met(actual, predicted)
                cm = confusion_matrix(actual, predicted)
                print('\n'.join([''.join(['{:4}'.format(item) for item in row]) for row in cm]))
                #confusionmatrix = np.matrix(cm)
                FP = cm.sum(axis=0) - np.diag(cm)
                FN = cm.sum(axis=1) - np.diag(cm)
                TP = np.diag(cm)
                TN = cm.sum() - (FP + FN + TP)
                print('False Positives\n {}'.format(FP))
                print('False Negetives\n {}'.format(FN))
                print('True Positives\n {}'.format(TP))
                print('True Negetives\n {}'.format(TN))
                TPR = TP/(TP+FN)
                print('Sensitivity \n {}'.format(TPR))
                TNR = TN/(TN+FP)
                print('Specificity \n {}'.format(TNR))
                Precision = TP/(TP+FP)
                print('Precision \n {}'.format(Precision))
                Recall = TP/(TP+FN)
                print('Recall \n {}'.format(Recall))
                Acc = (TP+TN)/(TP+TN+FP+FN)
                print('Áccuracy \n{}'.format(Acc))
                Fscore = 2*(Precision*Recall)/(Precision+Recall)
                print('FScore \n{}'.format(Fscore))
                k=cohen_kappa_score(actual, predicted)
                print('Çohen Kappa \n{}'.format(k))
                scores.append(accuracy)
        return scores

In [7]:
def activate(weights, inputs):
        activation = weights[-1]
        #print (len(weights)," : " ,len(inputs))
        for i in range(len(inputs)-1):
                activation += weights[i] * inputs[i]
        return activation

In [8]:
def transfer(activation):
        return 1.0 / (1.0 + exp(-activation))
 
# Forward propagate input to a network output
def forward_propagate(network, row):
        inputs = row
        for layer in network:
                new_inputs = []
                for neuron in layer:
                        activation = activate(neuron['weights'], inputs)
                        neuron['output'] = transfer(activation)
                        new_inputs.append(neuron['output'])
                inputs = new_inputs
        return inputs

In [9]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
        return output * (1.0 - output)
 
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
        for i in reversed(range(len(network))):
                layer = network[i]
                errors = list()
                if i != len(network)-1:
                        for j in range(len(layer)):
                                error = 0.0
                                for neuron in network[i + 1]:
                                        error += (neuron['weights'][j] * neuron['delta'])
                                errors.append(error)
                else:
                        for j in range(len(layer)):
                                neuron = layer[j]
                                errors.append(expected[j] - neuron['output'])
                for j in range(len(layer)):
                        neuron = layer[j]
                        neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
 

In [10]:
def update_weights(network, row, l_rate):
        for i in range(len(network)):
                inputs = row[:-1]                
                if i != 0:
                        inputs = [neuron['output'] for neuron in network[i - 1]]
                for neuron in network[i]:
                        for j in range(len(inputs)):
                                temp = l_rate * neuron['delta'] * inputs[j] + mu * neuron['prev'][j]
                                
                                neuron['weights'][j] += temp
                                #print("neuron weight{} \n".format(neuron['weights'][j]))
                                neuron['prev'][j] = temp
                        temp = l_rate * neuron['delta'] + mu * neuron['prev'][-1]
                        neuron['weights'][-1] += temp
                        neuron['prev'][-1] = temp
                                

In [11]:
def train_network(network, train, l_rate, n_epoch, n_outputs):
        for epoch in range(n_epoch):
                for row in train:
                        outputs = forward_propagate(network, row)
                        #print(network)
                        expected = [0 for i in range(n_outputs)]
                        expected[int(row[-1])] = 1
                        #print("expected row{}\n".format(expected))
                        backward_propagate_error(network, expected)
                        update_weights(network, row, l_rate)
 

In [12]:
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
        network = list()
        hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)], 'prev':[0 for i in range(n_inputs+1)]} for i in range(n_hidden)]        
        network.append(hidden_layer)
        hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)], 'prev':[0 for i in range(n_inputs+1)]} for i in range(n_hidden)]        
        network.append(hidden_layer)
        output_layer = [{'weights':[random() for i in range(n_hidden + 1)],'prev':[0 for i in range(n_hidden+1)]} for i in range(n_outputs)]
        network.append(output_layer)
        #print(network)
        return network

In [13]:
# Make a prediction with a network
def predict(network, row):
        outputs = forward_propagate(network, row)
        return outputs.index(max(outputs))
 
# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
        n_inputs = len(train[0]) - 1
        n_outputs = len(set([row[-1] for row in train]))
        network = initialize_network(n_inputs, n_hidden, n_outputs)
        train_network(network, train, l_rate, n_epoch, n_outputs)
        #print("network {}\n".format(network))
        predictions = list()
        for row in test:
                prediction = predict(network, row)
                predictions.append(prediction)
        return(predictions)
 

In [16]:
# Test Backprop on Seeds dataset
import pandas as pd
seed(1)
# load and prepare data
filename = 'Iris.csv'
dataset = pd.read_csv(filename)
dataset.head()

,0,1,2,3,4,5
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [17]:
cols = dataset.columns
features = cols[0:4]
labels = cols[4]
print(features)
print(labels)

Index(['0', '1', '2', '3'], dtype='object')
4


In [18]:
data_norm = pd.DataFrame(dataset)

for feature in features:
    dataset[feature] = (dataset[feature] - dataset[feature].mean())/dataset[feature].std()

#Show that should now have zero mean
print("Averages")
print(dataset.mean())

print("\n Deviations")
#Show that we have equal variance
print(pow(dataset.std(),2))

Averages
0    2.220446e-17
1   -1.457168e-15
2   -1.722511e-15
3   -2.043551e-15
4    1.198667e+00
dtype: float64

 Deviations
0    1.000000
1    1.000000
2    1.000000
3    1.000000
4    0.582414
dtype: float64


In [19]:
data_norm.head()

,0,1,2,3,4,5
0,-1.714797,-0.897674,1.028611,-1.336794,0.2,Iris-setosa
1,-1.691780,-1.139200,-0.124540,-1.336794,0.2,Iris-setosa
2,-1.668762,-1.380727,0.336720,-1.393470,0.2,Iris-setosa
3,-1.645745,-1.501490,0.106090,-1.280118,0.2,Iris-setosa
4,-1.622728,-1.018437,1.259242,-1.336794,0.2,Iris-setosa


In [20]:
X = data_norm.drop(['5'],axis=1)
X.head()

,0,1,2,3,4
0,-1.714797,-0.897674,1.028611,-1.336794,0.2
1,-1.691780,-1.139200,-0.124540,-1.336794,0.2
2,-1.668762,-1.380727,0.336720,-1.393470,0.2
3,-1.645745,-1.501490,0.106090,-1.280118,0.2
4,-1.622728,-1.018437,1.259242,-1.336794,0.2


In [21]:
y = data_norm[['5']]
y.head()

,5
0,Iris-setosa
1,Iris-setosa
2,Iris-setosa
3,Iris-setosa
4,Iris-setosa


In [22]:
from sklearn.preprocessing  import LabelEncoder

enc = LabelEncoder()
y.loc[:,['5']] = y.loc[:,['5']].apply(enc.fit_transform)
y.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,5
0,0
1,0
2,0
3,0
4,0


In [23]:
y['5'].value_counts()

2    50
1    50
0    50
Name: 5, dtype: int64

In [24]:
X_all = pd.concat([X,y],axis=1)
# Generate Training and Validation Sets
#X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.3)

# Convert to np arrays so that we can use with TensorFlow
X_train = np.array(X_all).astype(np.float32)
#y_train = np.array(y).astype(np.float32)

In [25]:
X_train.shape

(150, 6)

In [ ]:
y.shape

(150, 1)

In [26]:
n_folds = 5
l_rate = 0.1
mu=0.001
n_epoch = 1500
n_hidden = 2
scores = run_algorithm(X_train, back_propagation, n_folds, l_rate, n_epoch, n_hidden)

  11   0   0
   0   6   2
   0   0  11
False Positives
 [0 0 2]
False Negetives
 [0 2 0]
True Positives
 [11  6 11]
True Negetives
 [19 22 17]
Sensitivity 
 [1.   0.75 1.  ]
Specificity 
 [1.         1.         0.89473684]
Precision 
 [1.         1.         0.84615385]
Recall 
 [1.   0.75 1.  ]
Áccuracy 
[1.         0.93333333 0.93333333]
FScore 
[1.         0.85714286 0.91666667]
Çohen Kappa 
0.8979591836734694
  10   0   0
   0  10   1
   0   0   9
False Positives
 [0 0 1]
False Negetives
 [0 1 0]
True Positives
 [10 10  9]
True Negetives
 [20 19 20]
Sensitivity 
 [1.         0.90909091 1.        ]
Specificity 
 [1.         1.         0.95238095]
Precision 
 [1.  1.  0.9]
Recall 
 [1.         0.90909091 1.        ]
Áccuracy 
[1.         0.96666667 0.96666667]
FScore 
[1.         0.95238095 0.94736842]
Çohen Kappa 
0.95
   6   0   0
   7   3   0
   0   0  14
False Positives
 [7 0 0]
False Negetives
 [0 7 0]
True Positives
 [ 6  3 14]
True Negetives
 [17 20 16]
Sensitivity 
 [1.  0.3 1